In [ ]:

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os
dataset_path = "/content/drive/MyDrive/DermNet"
train_dir = os.path.join(dataset_path, "train")
test_dir = os.path.join(dataset_path, "test")

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Аугментации и нормализация
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [ ]:

train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
test_dataset = datasets.ImageFolder(test_dir, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Количество классов: {len(train_dataset.classes)}")
print(f"Пример классов: {train_dataset.classes}")

Количество классов: 23
Пример классов: ['Акне и розацеа', 'Актинический кератоз Базалиома и другие злокачественные новообразования', 'Атопический дерматит', 'Бородавки, моллюски и другие вирусные инфекции', 'Буллезная болезнь', 'Васкулит', 'Волчанка и другие заболевания соединительной ткани', 'Выпадение волос, алопеция и другие заболевания волосяного покрова', 'Герпес ВПЧ и другие ЗППП', 'Грибок ногтей и другие заболевания ногтей', 'Крапивница', 'Легкие заболевания и нарушения пигментации', 'Меланома, рак кожи, невусы и родинки', 'Опоясывающий лишай Кандидоз и другие грибковые инфекции', 'Псориаз, красный плоский лишай и связанные с ними заболевания', 'Себорейные кератозы и другие доброкачественные опухоли', 'Системное заболевание', 'Сосудистые опухоли', 'Целлюлит, импетиго и другие бактериальные инфекции', 'Чесотка, болезнь Лайма и другие инвазии и укусы', 'Экзантемы и лекарственные высыпания', 'Экзема', 'Ядовитый плющ и другие контактные дерматиты']


In [ ]:
# Загрузка и настройка модели
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))
model = model.to(device)

In [ ]:
# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

In [ ]:

def train_model(model, train_loader, test_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f"Эпоха {epoch+1}: Потери: {running_loss:.4f}, Точность на обучении: {accuracy:.4f}")
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        val_acc = val_correct / val_total
        print(f"           Точность на тесте: {val_acc:.4f}")
train_model(model, train_loader, test_loader, epochs=10)
model_save_path = "/content/drive/MyDrive/dermnet_model.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Модель сохранена по пути: {model_save_path}")

Эпоха 1: Потери: 399.3830, Точность на обучении: 0.3783
           Точность на тесте: 0.3238
Эпоха 2: Потери: 324.4625, Точность на обучении: 0.4725
           Точность на тесте: 0.4003
Эпоха 3: Потери: 281.4313, Точность на обучении: 0.5430
           Точность на тесте: 0.5272
Эпоха 4: Потери: 234.6953, Точность на обучении: 0.6075
           Точность на тесте: 0.5945
Эпоха 5: Потери: 195.0492, Точность на обучении: 0.6681
           Точность на тесте: 0.6549
Эпоха 6: Потери: 164.7127, Точность на обучении: 0.7190
           Точность на тесте: 0.6649
Эпоха 7: Потери: 126.7632, Точность на обучении: 0.7831
           Точность на тесте: 0.8048
Эпоха 8: Потери: 99.6232, Точность на обучении: 0.8308
           Точность на тесте: 0.8251
Эпоха 9: Потери: 78.4171, Точность на обучении: 0.8705
           Точность на тесте: 0.8808
Эпоха 10: Потери: 65.9043, Точность на обучении: 0.8890
           Точность на тесте: 0.9043
Модель сохранена по пути: /content/drive/MyDrive/dermnet_model.pth
